https://webnautes.tistory.com/1658

In [1]:
import pandas as pd
import random
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

import xgboost
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

if os.name == 'posix': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif os.name == 'nt': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

NameError: name 'plt' is not defined

# 시드 고정

In [ ]:
CFG = {
    'SEED':42
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED'])

# Load Data

In [6]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

display(train_data.head(3))
display(test_data.head(3))

,ID,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,TRAIN_00000,9,화요일,10,137,8.0,2.611124,0.0,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2
1,TRAIN_00001,11,화요일,6,438,13.0,3.209093,0.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
2,TRAIN_00002,8,일요일,6,1729,47.0,1.619597,0.0,0.0,0.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1


,ID,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지
0,TEST_00000,9,금요일,5,927,28.0,1.570654,19.625000,0.0,0.0,165.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,차도
1,TEST_00001,5,수요일,3,926,28.0,1.712457,21.444444,0.0,0.0,175.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,식당
2,TEST_00002,5,월요일,6,1437,33.0,0.447496,25.200000,0.0,0.0,290.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,주거지


In [10]:
test_data['요일'].unique()
train_data['요일'].unique()

array(['화요일', '일요일', '월요일', '목요일', '수요일', '토요일', '금요일'], dtype=object)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, train_label, test_size=0.2, random_state=42, stratify=train_label)

for i in [X_train, X_test, y_train, y_test]:
    print(len(i))

In [ ]:
display(y_test.value_counts()/sum(y_test.values)*100)
display(y_train.value_counts()/sum(y_train.values)*100)

# Pipe line

# Label Encoding

In [ ]:
ordinal_features = ['요일', '범죄발생지']

for features in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[features])
    train[features] = le.transform(train[features])
    X_train[features] = le.transform(X_train[features])
    X_test[features] = le.transform(X_test[features])
    for label in np.unique(test[features]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[features] = le.transform(test[features])

# Sample Weights

In [ ]:
y_train.value_counts()

In [ ]:
np.bincount(y_train)

In [ ]:
def CreateBalancedSampleWeights(y_train, largest_class_weight_coef):
    classes = y_train.unique()
    classes.sort()
    class_samples = np.bincount(y_train)
    total_samples = class_samples.sum()
    n_classes = len(class_samples)
    weights = total_samples / (n_classes * class_samples * 1.0)
    class_weight_dict = {key : value for (key, value) in zip(classes, weights)}
    class_weight_dict[classes[1]] = class_weight_dict[classes[1]] * largest_class_weight_coef
    sample_weights = [class_weight_dict[y] for y in y_train]

    return sample_weights

In [ ]:
train_sample_weight = CreateBalancedSampleWeights(y_train, largest_class_weight_coef=1)
train_sample_weight[:10]

# Optuna

In [ ]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(), )

study.optimize(lambda trial:objectigveXGB(trial, X_train, y_train, X_test), n_trials=50)